# [ASI Project] Weight Uncertainty in Neural Networks  
**Authors**: Miriam Lamari, Francesco Giannuzzo  


In [1]:
import csv
import math
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torchvision import datasets
from torchvision import transforms
from kaggle_secrets import UserSecretsClient
from torch.utils.data import DataLoader, random_split
import wandb

In [2]:
user_secrets = UserSecretsClient()
key = user_secrets.get_secret('wandb-api-key')

wandb.login(key=key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: francescogiannuzzo2002-fg (miriam-lamari2-eurecom) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Minibatches
**minibatch_weight**(batch_idx: int, num_batches: int)

In [3]:
def minibatch_weight(batch_idx: int, num_batches: int) -> float:
    return 2 ** (num_batches - batch_idx) / (2 ** num_batches - 1)#batch_idx)

## Variational Approximation

In [4]:
from typing import Any, Optional

import torch.nn as nn
from torch import Tensor


class BayesianModule(nn.Module):

    """Base class for BNN to enable certain behaviour."""

    def __init__(self):
        super().__init__()

    def kld(self, *args):
        raise NotImplementedError('BayesianModule::kld()')


def variational_approximator(model: nn.Module) -> nn.Module:

    def kl_divergence(self) -> Tensor:

        kl = 0
        for module in self.modules():
            if isinstance(module, BayesianModule):
                kl += module.kl_divergence

        return kl

    # add `kl_divergence` to the model
    setattr(model, 'kl_divergence', kl_divergence)

    def elbo(self,
             inputs: Tensor,
             targets: Tensor,
             criterion: Any,
             n_samples: int,
             w_complexity: Optional[float] = 1.0) -> Tensor:

        loss = 0
        for sample in range(n_samples):
            outputs = self(inputs)
            loss += criterion(outputs, targets)
            loss += self.kl_divergence() * w_complexity

        return loss / n_samples

    # add `elbo` to the model
    setattr(model, 'elbo', elbo)

    return model


## Scale Mixture Prior

In [5]:
import functools as ft

import torch
import torch.nn as nn
from torch import Tensor


class ScaleMixture(nn.Module):

    """Scale Mixture Prior.

    Section 3.3 of the 'Weight Uncertainty in Neural Networks' paper
    proposes the use of a Scale Mixture prior for use in variational
    inference - this being a fixed-form prior.

    The authors note that, should the parameters be allowed to adjust
    during training, the prior changes rapidly and attempts to capture
    the empirical distribution of the weights. As a result the prior
    learns to fit poor initial parameters and struggles to improve.
    """

    def __init__(self, pi: float, sigma1: float, sigma2: float) -> None:

        """Scale Mixture Prior.

        The authors of 'Weight Uncertainty in Neural Networks' note:

            sigma1 > sigma2:
                provides a heavier tail in the prior density than is
                seen in a plain Gaussian prior.
            sigma2 << 1.0:
                causes many of the weights to a priori tightly
                concentrate around zero.

        Parameters
        ----------
        pi : float
            Parameter used to scale the two Gaussian distributions.
        sigma1 : float
            Standard deviation of the first normal distribution.
        sigma2 : float
            Standard deviation of the second normal distribution.
        """

        super().__init__()

        self.pi = pi
        self.sigma1 = sigma1
        self.sigma2 = sigma2

        self.normal1 = torch.distributions.Normal(0, sigma1)
        self.normal2 = torch.distributions.Normal(0, sigma2)

    def log_prior(self, w: Tensor) -> Tensor:

        """Log Likelihood of the weight according to the prior.

        Calculates the log likelihood of the supplied weight given the
        prior distribution - the scale mixture of two Gaussians.

        Parameters
        ----------
        w : Tensor
            Weight to be used to calculate the log likelihood.

        Returns
        -------
        Tensor
            Log likelihood of the weights from the prior distribution.
        """

        likelihood_n1 = torch.exp(self.normal1.log_prob(w))
        likelihood_n2 = torch.exp(self.normal2.log_prob(w))

        p_scalemixture = self.pi * likelihood_n1 + (1 - self.pi) * likelihood_n2
        log_prob = torch.log(p_scalemixture).sum()

        return log_prob

## Gaussian Variational Inference

In [6]:
import numpy as np
import torch
import torch.nn as nn
from torch import Tensor


class GaussianVariational(nn.Module):
    def __init__(self, mu: Tensor, rho: Tensor) -> None:

        super().__init__()

        self.mu = nn.Parameter(mu)
        self.rho = nn.Parameter(rho)

        self.w = None
        self.sigma = None

        self.normal = torch.distributions.Normal(0, 1)

    def sample(self) -> Tensor:
        device = self.mu.device
        epsilon = self.normal.sample(self.mu.size()).to(device)
        self.sigma = torch.log(1 + torch.exp(self.rho)).to(device)
        self.w = self.mu + self.sigma * epsilon

        return self.w

    def log_posterior(self) -> Tensor:

        if self.w is None:
            raise ValueError('self.w must have a value.')

        log_const = np.log(np.sqrt(2 * np.pi))
        log_exp = ((self.w - self.mu) ** 2) / (2 * self.sigma ** 2)
        log_posterior = -log_const - torch.log(self.sigma) - log_exp

        return log_posterior.sum()

## Bayesian Linear Layer ##

In [7]:
from typing import Optional

import torch
import torch.nn.functional as F
from torch import Tensor

#from .base_bayesian import BayesianModule
#from .samplers.gaussian_variational import GaussianVariational
#from .samplers.scale_mixture import ScaleMixture


class BayesLinear(BayesianModule):

    """Bayesian Linear Layer.

    Implementation of a Bayesian Linear Layer as described in the
    'Weight Uncertainty in Neural Networks' paper.
    """

    def __init__(self,
                 in_features: int,
                 out_features: int,
                 prior_pi: Optional[float] = 0.5,
                 prior_sigma1: Optional[float] = 1.0,
                 prior_sigma2: Optional[float] = 0.0025) -> None:

        super().__init__()

        w_mu = torch.empty(out_features, in_features).uniform_(-0.2, 0.2)
        w_rho = torch.empty(out_features, in_features).uniform_(-5.0, -4.0)

        bias_mu = torch.empty(out_features).uniform_(-0.2, 0.2)
        bias_rho = torch.empty(out_features).uniform_(-5.0, -4.0)

        self.w_posterior = GaussianVariational(w_mu, w_rho)
        self.bias_posterior = GaussianVariational(bias_mu, bias_rho)

        self.w_prior = ScaleMixture(prior_pi, prior_sigma1, prior_sigma2)
        self.bias_prior = ScaleMixture(prior_pi, prior_sigma1, prior_sigma2)

        self.kl_divergence = 0.0

    def forward(self, x: Tensor) -> Tensor:

        w = self.w_posterior.sample()
        b = self.bias_posterior.sample()

        w_log_prior = self.w_prior.log_prior(w)
        b_log_prior = self.bias_prior.log_prior(b)

        w_log_posterior = self.w_posterior.log_posterior()
        b_log_posterior = self.bias_posterior.log_posterior()

        total_log_prior = w_log_prior + b_log_prior
        total_log_posterior = w_log_posterior + b_log_posterior
        self.kl_divergence = self.kld(total_log_prior, total_log_posterior)

        return F.linear(x, w, b)

    def kld(self, log_prior: Tensor, log_posterior: Tensor) -> Tensor:
        return log_posterior - log_prior

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {}

In [9]:
def train_wrapper():
    run = wandb.init(project="Project-ASI")

    return train_loop(
        learning_rate = wandb.config.learning_rate,
        prior_pi = wandb.config.prior_pi,
        prior_sigma1=wandb.config.prior_sigma1,
        prior_sigma2=wandb.config.prior_sigma2
    )

## Tuning Hyperparamters

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, random_split
from sklearn.metrics import mean_squared_error

def generate_data(n_samples=200):
    x = np.linspace(-0.5, 2.5, n_samples)
    eps = np.random.normal(0, 0.02, size=n_samples)
    y = x + 0.3 * np.sin(2 * np.pi * (x + eps)) + 0.3 * np.sin(4 * np.pi * (x + eps)) + eps
    x = torch.tensor(x, dtype=torch.float32).unsqueeze(1)
    y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)
    return x, y

In [11]:
# === TRAIN LOOP ===
def train_loop(learning_rate, prior_pi, prior_sigma1, prior_sigma2, epochs=100):
    run = wandb.init(project="Project-ASI")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    @variational_approximator
    class BayesianNetwork(nn.Module):
        def __init__(self, input_dim, output_dim):
            super().__init__()
            self.bl1 = BayesLinear(input_dim, 1200, prior_pi, prior_sigma1, prior_sigma2)
            self.bl2 = BayesLinear(1200, 1200, prior_pi, prior_sigma1, prior_sigma2)
            self.bl3 = BayesLinear(1200, output_dim, prior_pi, prior_sigma1, prior_sigma2)
    
        def forward(self, x):
            x = F.relu(self.bl1(x))
            x = F.relu(self.bl2(x))
            x = self.bl3(x)
            
            # Accumula KL divergence da tutti i layer
            self.kl_divergence = (
                self.bl1.kl_divergence +
                self.bl2.kl_divergence +
                self.bl3.kl_divergence
            )
            
            return x  # output: (batch_size, 2) → [mean, raw_variance]
        
    # Data prep
    x, y = generate_data()
    full_dataset = TensorDataset(x, y)
   
    test_size = int(0.2 * len(full_dataset))
    val_size = int(0.3 * (len(full_dataset) - test_size))
    train_size = len(full_dataset) - test_size - val_size
    
    train_set, val_set, test_set = random_split(full_dataset, [train_size, val_size, test_size])
    trainloader = DataLoader(train_set, batch_size=32, shuffle=True) #explain in the report
    valloader = DataLoader(val_set, batch_size=32)
    testloader = DataLoader(test_set, batch_size=32)

    # Model
    model = BayesianNetwork(1, 2).to(device)  # Output: [mean, logvar]
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.GaussianNLLLoss(full=True, reduction='sum')

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for batch_idx, (x_batch, y_batch) in enumerate(trainloader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            pi_weight = minibatch_weight(batch_idx=batch_idx, num_batches=len(trainloader))

            output = model(x_batch)
            mean = output[:, 0:1]
            variance = F.softplus(output[:, 1:2]) + 1e-6
            
            nll = criterion(mean, y_batch, variance)
            kld = model.kl_divergence  # updated during forward
            loss = nll + pi_weight * kld

            train_loss += loss.item()
            loss.backward()
            optimizer.step()

        # === Validation ===
        model.eval()
        val_loss = 0.0
        val_rmse = 0.0

        with torch.no_grad():
            for batch_idx, (x_batch, y_batch) in enumerate(valloader):
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)

                output = model(x_batch)
                mean = output[:, 0:1]
                variance = F.softplus(output[:, 1:2]) + 1e-6

                nll = criterion(mean, y_batch, variance)
                kld = model.kl_divergence
                loss = nll + pi_weight * kld

                val_loss += loss.item()
                val_rmse += torch.sum((mean - y_batch) ** 2).item()

        val_rmse = np.sqrt(val_rmse / len(val_set))

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.2f} | "
              f"Val Loss: {val_loss:.2f} | Val RMSE: {val_rmse:.4f}")

    metrics = {'train_loss' : train_loss, 'val_loss' : val_loss, 'val_rmse' :  val_rmse}
    wandb.log(metrics)

"""""
    # === Test set evaluation ===
    model.eval()
    test_preds = []
    test_targets = []

    with torch.no_grad():
        for x_batch, y_batch in testloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            output = model(x_batch)
            mean = output[:, 0:1]
            test_preds.append(mean.cpu())
            test_targets.append(y_batch.cpu())

    test_preds = torch.cat(test_preds)
    test_targets = torch.cat(test_targets)
    test_rmse = torch.sqrt(F.mse_loss(test_preds, test_targets)).item()
    print(f"Test RMSE: {test_rmse:.4f}")
"""

'""\n    # === Test set evaluation ===\n    model.eval()\n    test_preds = []\n    test_targets = []\n\n    with torch.no_grad():\n        for x_batch, y_batch in testloader:\n            x_batch, y_batch = x_batch.to(device), y_batch.to(device)\n            output = model(x_batch)\n            mean = output[:, 0:1]\n            test_preds.append(mean.cpu())\n            test_targets.append(y_batch.cpu())\n\n    test_preds = torch.cat(test_preds)\n    test_targets = torch.cat(test_targets)\n    test_rmse = torch.sqrt(F.mse_loss(test_preds, test_targets)).item()\n    print(f"Test RMSE: {test_rmse:.4f}")\n'

In [12]:
sweep_configuration = {
     "method": "grid",
     "metric": {"goal": "minimize", "name": "val_loss"},
     'name': "sweep-BBB-Gaussian_regression",
     "parameters": {
         "learning_rate": {'values': [1e-3, 1e-4]},
         "prior_pi": {'values': [0.25, 0.5]},
         "prior_sigma1": {'values': [1, math.exp(-1)]},
         "prior_sigma2": {'values': [math.exp(-6), math.exp(-7)]},
     },
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project="Project-ASI")
wandb.agent(sweep_id, function=train_wrapper);

Create sweep with ID: 6pjj1s0i
Sweep URL: https://wandb.ai/miriam-lamari2-eurecom/Project-ASI/sweeps/6pjj1s0i


wandb: Agent Starting Run: n7yh3yry with config:
wandb: 	learning_rate: 0.001
wandb: 	prior_pi: 0.25
wandb: 	prior_sigma1: 1
wandb: 	prior_sigma2: 0.0024787521766663585
wandb: WARNING Ignoring project 'Project-ASI' when running a sweep.
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250608_091747-n7yh3yry
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run sunny-sweep-1
wandb: ⭐️ View project at https://wandb.ai/miriam-lamari2-eurecom/Project-ASI
wandb: 🧹 View sweep at https://wandb.ai/miriam-lamari2-eurecom/Project-ASI/sweeps/6pjj1s0i
wandb: 🚀 View run at https://wandb.ai/miriam-lamari2-eurecom/Project-ASI/runs/n7yh3yry
wandb: WARNING Ignoring project 'Project-ASI' when running a sweep.
wandb: uploading output.log; uploading wandb-summary.json; uploading config.yaml
wandb:                                                                                
wandb: 🚀 View run sunny-sweep-1 at: https://wandb.ai/miri

Epoch 1/100 | Train Loss: 14940277.00 | Val Loss: 1982882.56 | Val RMSE: 3.9028
Epoch 2/100 | Train Loss: 14859162.50 | Val Loss: 1974179.81 | Val RMSE: 4.7342
Epoch 3/100 | Train Loss: 14791836.38 | Val Loss: 1965393.44 | Val RMSE: 4.7683
Epoch 4/100 | Train Loss: 14727640.38 | Val Loss: 1956439.50 | Val RMSE: 3.6601
Epoch 5/100 | Train Loss: 14664721.38 | Val Loss: 1948379.00 | Val RMSE: 2.6705
Epoch 6/100 | Train Loss: 14601707.62 | Val Loss: 1939686.38 | Val RMSE: 1.5529
Epoch 7/100 | Train Loss: 14532875.31 | Val Loss: 1930251.25 | Val RMSE: 1.2101
Epoch 8/100 | Train Loss: 14465265.19 | Val Loss: 1920989.12 | Val RMSE: 1.1063
Epoch 9/100 | Train Loss: 14396127.19 | Val Loss: 1912294.50 | Val RMSE: 0.9736
Epoch 10/100 | Train Loss: 14328198.00 | Val Loss: 1902370.44 | Val RMSE: 0.8563
Epoch 11/100 | Train Loss: 14251797.56 | Val Loss: 1892482.12 | Val RMSE: 0.5224
Epoch 12/100 | Train Loss: 14180437.06 | Val Loss: 1882174.50 | Val RMSE: 0.8456
Epoch 13/100 | Train Loss: 14098597.7

wandb: uploading output.log; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb: train_loss ▁
wandb:   val_loss ▁
wandb:   val_rmse ▁
wandb: 
wandb: Run summary:
wandb: train_loss 9402503.9375
wandb:   val_loss 1252236.625
wandb:   val_rmse 0.36418
wandb: 
wandb: 🚀 View run sunny-sweep-1 at: https://wandb.ai/miriam-lamari2-eurecom/Project-ASI/runs/n7yh3yry
wandb: ⭐️ View project at: https://wandb.ai/miriam-lamari2-eurecom/Project-ASI
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250608_091748-n7yh3yry/logs
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: opahk6sc with config:
wandb: 	learning_rate: 0.001
wandb: 	prior_pi: 0.25
wandb: 	prior_sigma1: 1
wandb: 	prior_sigma2: 0.0009118819655545162
wandb: WARNING Ignoring project 'Project-ASI' when running a sweep.
wandb: Tracking run with wa

Epoch 1/100 | Train Loss: 15312938.88 | Val Loss: 2037285.12 | Val RMSE: 1.8420
Epoch 2/100 | Train Loss: 15279642.62 | Val Loss: 2033380.56 | Val RMSE: 2.1400
Epoch 3/100 | Train Loss: 15246114.62 | Val Loss: 2029604.12 | Val RMSE: 3.0930
Epoch 4/100 | Train Loss: 15220165.44 | Val Loss: 2026001.00 | Val RMSE: 2.6451
Epoch 5/100 | Train Loss: 15192272.94 | Val Loss: 2022939.38 | Val RMSE: 2.0092
Epoch 6/100 | Train Loss: 15165617.75 | Val Loss: 2019779.62 | Val RMSE: 1.2148
Epoch 7/100 | Train Loss: 15140811.56 | Val Loss: 2015970.62 | Val RMSE: 0.7178
Epoch 8/100 | Train Loss: 15111967.75 | Val Loss: 2012445.81 | Val RMSE: 0.6369
Epoch 9/100 | Train Loss: 15088758.06 | Val Loss: 2008874.56 | Val RMSE: 0.6955
Epoch 10/100 | Train Loss: 15059880.31 | Val Loss: 2005151.19 | Val RMSE: 0.4960
Epoch 11/100 | Train Loss: 15033962.31 | Val Loss: 2001482.94 | Val RMSE: 0.4573
Epoch 12/100 | Train Loss: 15004777.62 | Val Loss: 1998788.31 | Val RMSE: 0.7335
Epoch 13/100 | Train Loss: 14981109.1

...

wandb: uploading output.log; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb: train_loss ▁
wandb:   val_loss ▁
wandb:   val_rmse ▁
wandb: 
wandb: Run summary:
wandb: train_loss 13212914.75
wandb:   val_loss 1761577.8125
wandb:   val_rmse 0.34293
wandb: 
wandb: 🚀 View run stilted-sweep-14 at: https://wandb.ai/miriam-lamari2-eurecom/Project-ASI/runs/xrzogen3
wandb: ⭐️ View project at: https://wandb.ai/miriam-lamari2-eurecom/Project-ASI
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250608_092327-xrzogen3/logs
wandb: Agent Starting Run: 33puzsjj with config:
wandb: 	learning_rate: 0.0001
wandb: 	prior_pi: 0.5
wandb: 	prior_sigma1: 0.36787944117144233
wandb: 	prior_sigma2: 0.0024787521766663585
wandb: WARNING Ignoring project 'Project-ASI' when running a sweep.
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is

Epoch 1/100 | Train Loss: 10441983.56 | Val Loss: 1391242.62 | Val RMSE: 1.2844
Epoch 2/100 | Train Loss: 10434884.12 | Val Loss: 1390803.31 | Val RMSE: 0.5995
Epoch 3/100 | Train Loss: 10428824.56 | Val Loss: 1390519.50 | Val RMSE: 0.6479
Epoch 4/100 | Train Loss: 10424312.19 | Val Loss: 1389241.75 | Val RMSE: 1.0744
Epoch 5/100 | Train Loss: 10420045.31 | Val Loss: 1389051.88 | Val RMSE: 1.1253
Epoch 6/100 | Train Loss: 10418816.00 | Val Loss: 1388649.31 | Val RMSE: 0.7851
Epoch 7/100 | Train Loss: 10413674.62 | Val Loss: 1387759.62 | Val RMSE: 0.5266
Epoch 8/100 | Train Loss: 10408506.94 | Val Loss: 1387419.81 | Val RMSE: 0.4605
Epoch 9/100 | Train Loss: 10403132.81 | Val Loss: 1386740.12 | Val RMSE: 0.3247
Epoch 10/100 | Train Loss: 10400569.19 | Val Loss: 1386053.44 | Val RMSE: 0.6764
Epoch 11/100 | Train Loss: 10395323.44 | Val Loss: 1385397.31 | Val RMSE: 0.3651
Epoch 12/100 | Train Loss: 10391851.00 | Val Loss: 1385208.19 | Val RMSE: 0.4159
Epoch 13/100 | Train Loss: 10388402.5

wandb: uploading wandb-summary.json; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb: train_loss ▁
wandb:   val_loss ▁
wandb:   val_rmse ▁
wandb: 
wandb: Run summary:
wandb: train_loss 10009559.125
wandb:   val_loss 1333982.875
wandb:   val_rmse 0.36388
wandb: 
wandb: 🚀 View run denim-sweep-15 at: https://wandb.ai/miriam-lamari2-eurecom/Project-ASI/runs/33puzsjj
wandb: ⭐️ View project at: https://wandb.ai/miriam-lamari2-eurecom/Project-ASI
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250608_092348-33puzsjj/logs
wandb: Agent Starting Run: o4cuac4v with config:
wandb: 	learning_rate: 0.0001
wandb: 	prior_pi: 0.5
wandb: 	prior_sigma1: 0.36787944117144233
wandb: 	prior_sigma2: 0.0009118819655545162
wandb: WARNING Ignoring project 'Project-ASI' when running a sweep.
wandb: Tracking run with wandb version 0.19.9
wandb: Run d

Epoch 1/100 | Train Loss: 10651524.69 | Val Loss: 1419626.00 | Val RMSE: 0.4507
Epoch 2/100 | Train Loss: 10644025.56 | Val Loss: 1419517.94 | Val RMSE: 0.5636
Epoch 3/100 | Train Loss: 10646265.75 | Val Loss: 1419013.94 | Val RMSE: 0.5183
Epoch 4/100 | Train Loss: 10642503.12 | Val Loss: 1418808.81 | Val RMSE: 0.3315
Epoch 5/100 | Train Loss: 10638602.44 | Val Loss: 1418039.69 | Val RMSE: 0.4204
Epoch 6/100 | Train Loss: 10637075.12 | Val Loss: 1418135.38 | Val RMSE: 0.3864
Epoch 7/100 | Train Loss: 10634814.88 | Val Loss: 1417350.12 | Val RMSE: 0.3275
Epoch 8/100 | Train Loss: 10634539.69 | Val Loss: 1417331.00 | Val RMSE: 0.4271
Epoch 9/100 | Train Loss: 10629762.00 | Val Loss: 1416995.56 | Val RMSE: 0.4139
Epoch 10/100 | Train Loss: 10627875.06 | Val Loss: 1417011.31 | Val RMSE: 0.4643
Epoch 11/100 | Train Loss: 10626108.94 | Val Loss: 1416351.88 | Val RMSE: 0.3194
Epoch 12/100 | Train Loss: 10623092.38 | Val Loss: 1416227.81 | Val RMSE: 0.5110
Epoch 13/100 | Train Loss: 10620100.5

wandb: uploading output.log; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb: train_loss ▁
wandb:   val_loss ▁
wandb:   val_rmse ▁
wandb: 
wandb: Run summary:
wandb: train_loss 10455095.4375
wandb:   val_loss 1393360.0
wandb:   val_rmse 0.32271
wandb: 
wandb: 🚀 View run fine-sweep-16 at: https://wandb.ai/miriam-lamari2-eurecom/Project-ASI/runs/o4cuac4v
wandb: ⭐️ View project at: https://wandb.ai/miriam-lamari2-eurecom/Project-ASI
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250608_092414-o4cuac4v/logs
wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [13]:
#wandb.init(project="Project-ASI")

#train_loop(
#    learning_rate = best_hyperparameters['learning_rate'],
#    prior_pi = best_hyperparameters['prior_pi'],
#    prior_sigma1 = best_hyperparameters['prior_sigma1'],
#    prior_sigma2 = best_hyperparameters['prior_sigma2'],
#    epochs = 2
#)